# 02. Moving to a Framework: PydanticAI

In tutorial 01 we wired up a bare-bones agent manually. This lesson upgrades the workflow by adopting **[PydanticAI](https://github.com/pydantic/pydantic-ai)**, a lightweight agent framework that keeps prompts, tools, and validation in one place. We'll keep the examples compact so you can focus on the core ideas.

## What you'll learn
- How to bootstrap an agent with `Agent` and a hosted model.
- How to register tools and inject dependencies that give your model superpowers.
- How to add structured outputs, guardrails (validation + redaction), and retries.
- How to stream responses and log the conversation with Logfire.
- How to glue everything together in a realistic concierge use-case.

## 0. Requirements
1. Install dependencies locally (already present in this course environment). If you're on your own machine run:
   ```bash
   pip install pydantic-ai logfire ipykernel
   ```
2. Provide an API key for the provider you want to use (OpenAI works great). In a terminal run `export OPENAI_API_KEY=...` before launching Jupyter.
3. Optional but recommended: create a free [Logfire](https://logfire.pydantic.dev/) account for rich traces.

In [ ]:
import os

if "OPENAI_API_KEY" not in os.environ:
    print("⚠️  Set the OPENAI_API_KEY environment variable before talking to a hosted model.")
else:
    print("API key detected ✅")

## 1. Meet `Agent`
PydanticAI wraps model calls inside an `Agent`. You configure the model, give it a concise system prompt, and call `run_sync` to execute a turn.

We'll use a concierge persona throughout the notebook.

In [ ]:
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel

concierge = Agent(
    model=OpenAIModel(model="gpt-4o-mini"),
    system_prompt="You are a friendly city concierge. Keep replies short, positive, and specific to Berlin.",
)

intro = concierge.run_sync("Welcome a visitor to Berlin in one sentence.")
print(intro.response_text)

`Agent` accepts the model backend (OpenAI, Anthropic, etc.), a system prompt, and optional extras. `run_sync` sends the user message, waits for the model, and returns a `RunResult` with metadata like `response_text`, tool traces, and tokens.

## 2. Tools + Dependencies: Local Concierge Use-Case
Agents become useful once they can call real data. We'll model a concierge that knows Berlin restaurants and events stored in plain Python dictionaries.

PydanticAI lets you declare a dependency schema (`deps_type`) and register tool functions with the `@agent.tool` decorator. Tools receive a `RunContext` that exposes the dependencies plus request metadata.

In [ ]:
from typing import Dict, List

from pydantic import BaseModel

class ConciergeDeps(BaseModel):
    events: Dict[str, List[str]]
    restaurants: Dict[str, List[str]]

    def lookup(self, city: str) -> dict:
        key = city.lower()
        return {
            "events": self.events.get(key, []),
            "restaurants": self.restaurants.get(key, []),
        }

berlin_data = ConciergeDeps(
    events={
        "berlin": ["Saturday street food market in Kreuzberg", "Museum Island late-night opening", "Spree sunset boat cruise"],
    },
    restaurants={
        "berlin": ["Five Elephant Coffee Roastery", "Markthalle Neun vendors", "Mustafas Gemüse Kebap"],
    },
)

In [ ]:
from pydantic_ai import RunContext

guide = Agent(
    model=OpenAIModel(model="gpt-4o-mini"),
    system_prompt=(
        "You are a Berlin concierge. Use the `fetch_local_options` tool before planning a day so you stay factual. "
        "Summaries should include morning, afternoon, and evening suggestions."
    ),
    deps_type=ConciergeDeps,
)

@guide.tool
def fetch_local_options(ctx: RunContext[ConciergeDeps], city: str) -> dict:
    """Return curated restaurants and events for the requested city."""
    return ctx.deps.lookup(city)

day_plan = guide.run_sync(
    "Plan a Saturday in Berlin for a family with teenagers. Include meals and activities.",
    deps=berlin_data,
)
print(day_plan.response_text)

During the run the model can call `fetch_local_options`. The framework handles JSON arguments, dependency injection, and logging. Tool outputs are automatically threaded back into the model's context so later messages can reference them.

## 3. Model retry + Logfire tracing
Real traffic can be noisy: models may time out or return malformed JSON. `RetryPolicy` replays a call with the same inputs. Combining it with Logfire gives you deep observability—every attempt (and tool call) shows up in the Logfire UI.

In [ ]:
import logfire
from pydantic_ai import RetryPolicy

logfire.configure(send_to_logfire=False)  # keep traces local while experimenting

retry_policy = RetryPolicy(max_attempts=3)

with logfire.span("concierge-run"):
    winter_plan = guide.run_sync(
        "It's February and cold. Suggest an indoor-focused Berlin Saturday.",
        deps=berlin_data,
        retry=retry_policy,
    )

print(winter_plan.response_text)

## 4. Streaming replies
LLMs can stream partial text so you don't block users. `run_stream_sync` returns an iterator; use `text_stream()` to yield deltas as soon as they arrive.

In [ ]:
with guide.run_stream_sync(
    "Stream a warm two-sentence welcome for someone arriving in Berlin this evening.",
    deps=berlin_data,
) as stream:
    for token in stream.text_stream():
        print(token, end="", flush=True)

## 5. Guardrails: validation + redaction
Structured results reduce hallucinations. Pass a Pydantic model as `result_type` and the agent will coerce the response into that schema (retrying on validation errors). You can mark sensitive fields with `Redact` so that logs mask them automatically.

In [ ]:
from typing import Literal, Optional

from pydantic import BaseModel, EmailStr, Field, PositiveInt
from pydantic_ai import Redact

class DinnerReservation(BaseModel):
    guest_name: str = Field(max_length=60)
    guest_email: EmailStr = Redact()
    party_size: PositiveInt = Field(le=8)
    preferred_time: Literal["17:00", "18:00", "19:00", "20:00"]
    special_requests: Optional[str] = Field(default=None, max_length=200)

reservations = Agent(
    model=OpenAIModel(model="gpt-4o-mini"),
    system_prompt=(
        "Extract structured dinner reservations from chatty guest messages. "
        "If details are missing, make a polite best guess."
    ),
    result_type=DinnerReservation,
)

guest_message = """
Hi! I'm Alex visiting with 3 coworkers this Friday. Could you book something cozy around 7pm?
My email is alex@example.com and we'd love a vegetarian-friendly spot.
"""

reservation = reservations.run_sync(guest_message)

print(reservation.data.model_dump())
print("Shown in logs as:", reservation.data.model_dump(mode="json"))

Redacted fields still exist in `reservation.data`, but when Logfire (or standard logging) serializes the object the value is replaced with `***`.

## 6. Next steps
- Swap the dictionaries with a real API client or database dependency.
- Add more tools—weather lookup, transit times, ticket booking—and observe how the agent decides between them.
- Combine streaming with a UI (FastAPI, Streamlit) to deliver real-time concierge experiences.
- Explore advanced features like parallel tool calls, memory stores, and evaluation notebooks.